In [0]:
#from google.colab import files

#Subir archivo
#uploaded = files.upload() 

In [0]:
! git clone https://github.com/Duchanoy/DeepDamper.git


Cloning into 'DeepDamper'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 33 (delta 12), reused 24 (delta 9), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [0]:
%cd DeepDamper



/content/DeepDamper


In [0]:
!git config --global user.email "duch97@hotmail.com"
!git config --global user.name "Duchanoy"
!git remote rm origin
!git remote add origin https://token@github.com/Duchanoy/DeepDamper.git

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import json
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import os

In [0]:
def read_json(name):
    #print(name)
    with open(name) as data_file:    
            data = json.load(data_file)
    return data


In [0]:

def data_proces(name,Deface=100,NumSam=200):
    datos=read_json(name)
    entradas=read_json('_'.join(('inp',name)))
    train=np.array([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]])
    referencia=[0]
    for key in datos.keys():
        #print(len(datos[key]))
        for k,exp in enumerate(datos[key]):
            VelC=exp["velocity"]
            forceC=exp["force"]
            VelCj=[round(elem, 1) for elem in VelC]
            Conj=set(VelCj)
            infbar=100
            for elem in list(Conj):
                indices=[i for i,x in enumerate(VelCj) if x==elem]
                for ind in range(len(indices)-1):
                    if forceC[indices[ind]] == forceC[indices[ind+1]] and indices[ind]> 120:
                        reference = [indices[ind],indices[ind+1]]
                        break
                    elif abs(forceC[indices[ind]]-forceC[indices[ind+1]])<infbar and indices[ind+1]-indices[ind]>20 and indices[ind+1]-indices[ind]<200 and indices[ind]> 120:
                        tempref = [indices[ind],indices[ind+1]]
                        infbar = abs(forceC[indices[ind]]-forceC[indices[ind+1]])
                if 'reference' in locals():
                    break
            
            if 'reference' in locals():    
                Deface = reference[0]
                NumSam = reference[1] - reference[0]
                del reference
            else:
                Deface = tempref[0]
                NumSam = tempref[1] - tempref[0]
                del tempref
            referencia.append(referencia[-1]+NumSam+1)
            for element in range(NumSam+1):
                caso=list()
                caso.append(entradas[key]["Ap"])
                caso.append(entradas[key]["Ag"])
                caso.append(entradas[key]["h"])
                caso.append(entradas[key]["L"])
                caso.append(entradas[key]["viscocidad"])
                caso.append(entradas[key]["Ty"][k]/1000)
                caso.append((exp["velocity"][element+Deface])*0.001)
                caso.append(exp["displace"][element+Deface])
                caso.append(exp["force"][element+Deface])
                casonp=np.array(caso)
                #print(len(casonp))
                train=np.append(train,[casonp],axis=0)
            
    return train[2:][:],referencia



In [0]:
def Target_C(data):
    Arn=(data[0]+((data[1])/2))
    Fn=((12*(data[4])*Arn*(data[6]/10))/((data[1])*data[2]*data[2]))
    Fn=Fn*(data[0]*data[3])
    Fty=data[8]-Fn
    c=((Fty*data[2])/(data[3]*data[0]*data[5]*1000))
    return Fty


In [0]:
def force_limit(data):
    Arn=(data[0]+((data[1])/2))
    Fn=((12*(data[4])*Arn*(data[6]))/((data[1])*data[2]*data[2]))
    Fn=Fn*(data[0]*data[3])
    #c=2.07+((12*data[4]*data[0]*data[6])/((12*data[4]*data[0]*data[6])+(0.4*data[1]*data[2]*data[5])))
    c=1.07
    Fty= (c*data[3]*data[0]*data[5]*1000)/(data[2])
    fr=Fn+Fty
    #print(fr)
    #print(data[8])
    return fr

In [0]:
train,refer=data_proces("twofluidexp.json",Deface=120,NumSam=260)
#rint(train.shape)
#print(train[1:4])
np.random.shuffle(train)
#train[1:4,8:9].shape
#print(train[20][])
#print(refer)

In [0]:
Train=np.array(train)[:,:8].astype("float32")
Target=np.array(train)[:,8:9].astype("float32")

In [0]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

Arquitectura de la red usando el modelo de keras  por medio de objetos 


In [0]:
class DeepDamper(tf.keras.Model):
  def __init__(self):
    super(DeepDamper, self).__init__()
    self.HL1 = tf.keras.layers.Dense(15, input_shape=(8,) , activation='relu')
    self.HL2 = tf.keras.layers.Dense(15, activation = 'relu')
    self.HL3 = tf.keras.layers.Dense(15, activation = 'relu')
    self.out = tf.keras.layers.Dense(1)
    # Seleccionar una funcion de error, optimizador y metricas de evaluacion 
    self.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.MeanSquaredError())

  def call(self, inputs):
    x = self.HL1(inputs)
    x = self.HL2(x)
    x = self.HL3(x)
    x = self.out(x)
    return x


In [0]:
DamperModel = DeepDamper()
DamperModel.fit(x=Train, y=Target,batch_size =20, epochs=10000,callbacks=[cp_callback])

Train on 13333 samples
Epoch 1/10000
13300/13333 [============================>.] - ETA: 0s - loss: 98.6895
Epoch 00001: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 4s 285us/sample - loss: 98.6656
Epoch 2/10000
13260/13333 [============================>.] - ETA: 0s - loss: 93.0030
Epoch 00002: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 2s 146us/sample - loss: 92.9588
Epoch 3/10000
12920/13333 [============================>.] - ETA: 0s - loss: 91.4454
Epoch 00003: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 2s 132us/sample - loss: 91.2878
Epoch 4/10000
13200/13333 [============================>.] - ETA: 0s - loss: 90.9119
Epoch 00004: saving model to training_1/cp.ckpt
13333/13333 [==============================] - 2s 134us/sample - loss: 90.9388
Epoch 5/10000
12920/13333 [============================>.] - ETA: 0s - loss: 90.5757
Epoch 00005: saving model to training_1/cp.c

In [0]:
DamperModel.save('my_net') 

In [0]:
!git add -A
!git commit -m "Se agrego entrenamiento1"
!git push -u origin master